In [88]:
import pandas as pd
from datetime import date, timedelta
import numpy as np

In [89]:
""""
Aggregate the data so we have a single balance for each day already in the dataset, for each account
Scaffold the data so each account has a row between 31st Jan and 14th Feb (hint)
Make sure new rows have a null in the Transaction Value field
Create a parameter so a particular date can be selected
Filter to just this date
Output the data - making it clear which date is being filtered to
"""

'"\nAggregate the data so we have a single balance for each day already in the dataset, for each account\nScaffold the data so each account has a row between 31st Jan and 14th Feb (hint)\nMake sure new rows have a null in the Transaction Value field\nCreate a parameter so a particular date can be selected\nFilter to just this date\nOutput the data - making it clear which date is being filtered to\n'

In [90]:
#Import the data
df = pd.read_csv('Account Statements.csv')
df['Balance Date'] = pd.to_datetime(df['Balance Date'], dayfirst = True)

In [91]:
#Aggregate the data
df = df.groupby(by= ['Account Number', 'Balance Date' ])['Balance', 'Transaction Value'].sum()
df = df.reset_index()
df = df.sort_values(['Account Number','Balance Date'])
df

C:\Users\Dell\AppData\Local\Temp\ipykernel_10848\3414675.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(by= ['Account Number', 'Balance Date' ])['Balance', 'Transaction Value'].sum()


In [93]:
#Create a list of dates from the 31st of Jan to 14th of Feb. 
sdate = date(2023,1,31)
edate = date(2023,2,15)

all_dates = pd.date_range(sdate,edate-timedelta(days=1),freq='d').tolist()

In [128]:
#Scaffolding Function

def scaffolding(df,account_number): 
    sca_df = df[df['Account Number']==account_number]
    dates = list(set(all_dates) -set(sca_df['Balance Date'].tolist())) #Subtract 'all_dates' list from the list of dates in the dataframe for that account number
    dates.sort() 
    for date in dates: #Loop through the dates that we want to add.
        sca_df = sca_df.sort_values(['Balance Date'])
        df2 = {'Account Number': account_number, 
               'Balance Date': date,
               'Balance': sca_df[sca_df['Balance Date'] < date].iloc[-1]['Balance'], #Get the balance from the previous day 
                'Transaction Value': np.nan}
        df2 = pd.DataFrame([df2])
        sca_df = pd.concat([sca_df,df2],axis=0, ignore_index = True) #concat the dataframe with the new row
    return sca_df

In [129]:
#Get unique list of all accounts then loop through, apply the scaffolding function to each account number

all_accounts = df['Account Number'].unique().tolist()
final_df = pd.DataFrame(columns = ['Account Number', 'Balance Date', 'Balance', 'Transaction Value'])
for account in all_accounts: 
    scaffolded = scaffolding(df,account)
    final_df = pd.concat([final_df,scaffolded])

In [136]:
#Get user input and filter the final dataframe to that date

user_input = input("Enter a date (YYYY-MM-DD): ")
input_date = pd.to_datetime(user_input)

output = final_df[final_df['Balance Date'] == input_date]

output.to_csv('output.csv')